# Importing necessary Libraries

In [12]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [13]:
df = pd.read_csv("insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


### Preparing X and Y variables

In [14]:
X = df.drop(labels=['expenses'],axis=1)
y = df[['expenses']]

### Creating Data Transformation Pipeline

#### Creating Pipeline with Column Transformer

In [15]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, categorical_cols),
         ("StandardScaler", numeric_transformer, numerical_cols),        
    ]
)


In [16]:
print(categorical_cols)
print(numerical_cols)

Index(['sex', 'smoker', 'region'], dtype='object')
Index(['age', 'bmi', 'children'], dtype='object')


In [17]:
X = preprocessor.fit_transform(X)

### Train Test Split

In [18]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape


((1070, 11), (268, 11))

In [19]:
X.shape

(1338, 11)

### Model Training Baseline models

In [20]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

### Training Various models

In [21]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 6106.2929
- Mean Absolute Error: 4205.8634
- R2 Score: 0.7417
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5800.2322
- Mean Absolute Error: 4179.5421
- R2 Score: 0.7833


Lasso
Model performance for Training set
- Root Mean Squared Error: 6105.2476
- Mean Absolute Error: 4208.0337
- R2 Score: 0.7418
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5797.2951
- Mean Absolute Error: 4182.3918
- R2 Score: 0.7835


Ridge
Model performance for Training set
- Root Mean Squared Error: 6105.3089
- Mean Absolute Error: 4212.3760
- R2 Score: 0.7417
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5798.5673
- Mean Absolute Error: 4187.2631
- R2 Score: 0.7834


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 4770.4644
- Mean Absolute Error: 2857.6957


C:\Users\Dell\AppData\Local\Temp\ipykernel_12596\3127594056.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train) # Train model


Random Forest Regressor
Model performance for Training set
- Root Mean Squared Error: 1899.2027
- Mean Absolute Error: 1041.9028
- R2 Score: 0.9750
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4626.0304
- Mean Absolute Error: 2512.1046
- R2 Score: 0.8622


XGBRegressor
Model performance for Training set
- Root Mean Squared Error: 985.9242
- Mean Absolute Error: 533.1644
- R2 Score: 0.9933
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5025.5880
- Mean Absolute Error: 2903.8289
- R2 Score: 0.8373


CatBoosting Regressor
Model performance for Training set
- Root Mean Squared Error: 2699.3026
- Mean Absolute Error: 1512.7002
- R2 Score: 0.9495
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4557.2992
- Mean Absolute Error: 2533.2229
- R2 Score: 0.8662


AdaBoost Regressor
Model performance for Training set
- Root Mean Squared Error: 4940.2352
- Mean Ab

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Results

In [22]:
df_results = pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
df_results


,Model Name,R2_Score
7,CatBoosting Regressor,0.866221
5,Random Forest Regressor,0.862156
6,XGBRegressor,0.837316
8,AdaBoost Regressor,0.818827
1,Lasso,0.783518
2,Ridge,0.783422
0,Linear Regression,0.783298
3,K-Neighbors Regressor,0.768453
4,Decision Tree,0.695019
